In [1]:
import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
import matplotlib.pyplot as plt
import glob

In [2]:
files = glob.glob('../files/S001/*.edf')

In [3]:
len(files)

14

In [4]:
raws = []

for i in range(3, len(files)):
    current_file = files[i]
    r = read_raw_edf(current_file, preload=True, stim_channel='auto')
    events, _ = mne.events_from_annotations(r)
    if i % 3 == 0:
        new_labels_events = {1:'rest', 2:'action_hand', 3:'action_feet'} # action
    else:
        new_labels_events = {1:'rest', 2:'imagine_hand', 3:'imagine_feet'} # imagine
    new_annot = mne.annotations_from_events(events=events, event_desc=new_labels_events, sfreq=r.info['sfreq'], orig_time=r.info['meas_date'])
    r.set_annotations(new_annot)
    raws.append(r)
    
raw_obj = concatenate_raws(raws)

Extracting EDF parameters from /mnt/nfs/homes/oel-ayad/total-perspective-vortex/files/S001/S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /mnt/nfs/homes/oel-ayad/total-perspective-vortex/files/S001/S001R09.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /mnt/nfs/homes/oel-ayad/total-perspective-vortex/files/S001/S001R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /mnt/nfs/homes/oel-ayad/total-perspective-vortex/files/S001/S001R11.edf...
EDF file 

In [5]:
original_raw = raw_obj.copy()

In [6]:
events, event_dict = mne.events_from_annotations(original_raw)
print(original_raw.info)
print(event_dict)

Used Annotations descriptions: ['action_feet', 'action_hand', 'imagine_feet', 'imagine_hand', 'rest']
<Info | 7 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 80.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
>
{'action_feet': 1, 'action_hand': 2, 'imagine_feet': 3, 'imagine_hand': 4, 'rest': 5}


In [7]:
picks = mne.pick_types(original_raw.info, meg=True, eeg=True, stim=False, eog=False, exclude='bads')

In [8]:
low_cutoff = 10
high_cutoff = 33
original_raw.filter(low_cutoff, high_cutoff, fir_design='firwin')

Filtering raw data in 11 contiguous segments
Setting up band-pass filter from 10 - 33 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 10.00
- Lower transition bandwidth: 2.50 Hz (-6 dB cutoff frequency: 8.75 Hz)
- Upper passband edge: 33.00 Hz
- Upper transition bandwidth: 8.25 Hz (-6 dB cutoff frequency: 37.12 Hz)
- Filter length: 213 samples (1.331 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


<RawEDF | S001R10.edf, 64 x 209760 (1311.0 s), ~102.5 MB, data loaded>

In [9]:
common_spatial_pattern = original_raw.copy()

In [11]:
tmin = -0.5  # Time before event in seconds
tmax = 0.5   # Time after event in seconds
epochs = mne.Epochs(original_raw, events, event_dict, tmin, tmax, baseline=None, preload=True)
epochs

Not setting metadata
301 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 301 events and 161 original time points ...
11 bad epochs dropped


Number of events,290
Events,action_feet: 30action_hand: 30imagine_feet: 45imagine_hand: 45rest: 140
Time range,-0.500 – 0.500 s
Baseline,off


In [13]:
mne.time_frequency.tfr.cwt(epochs, epochs)

AttributeError: 'Epochs' object has no attribute 'shape'

Time-frequency techniques provide promising results for non-stationary EEG signals due to the best time-frequency resolutions [94]. For EEG analysis, short-time Fourier transform (STFT) is used for time-frequency resolution with compromised [95]. Wavelet Transform (WT) was developed for feature extraction in 1984 by Morlet and Grossman [96]. Wavelet transform (WT) permits better flexibility, and features can vary over different time scales and is a very promising method for feature extraction for BCI systems [97]. The WT offers the best results as compared to STFT [98]. In the continuous wavelets transform (CWT) approach, the wavelet is changed effortlessly over the scale, and calculating wavelet coefficients for each possible scale could require substantial effort. 


https://www.sciencedirect.com/science/article/pii/S2665917423001599